In [5]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [73]:
import requests
import json
import pandas as pd
import plotly.express as px
from lxml import etree
from zhconv import convert

def translate(string):
    return convert(string, 'zh-cn')

def format_code(code):
    code = code.replace('.HK','')
    code = '{:0>5}'.format(code)
    return code


def get_token():
    url_o='https://sc.hkex.com.hk/TuniS/www.hkex.com.hk/Market-Data/Securities-Prices/Equities?sc_lang=zh-HK'
    r = requests.get(url_o)
    token = etree.HTML(r.text).xpath('//*[contains(text(), "Base64-AES-Encrypted-Token")]')[0].text.split(';')[3].split()[1].replace('"','')
    return token


def get_industry_df():
    r = requests.get('https://static03.hket.com/data-lake/p/industry/industry-data.json')
    df = pd.DataFrame(json.loads(r.text))
    df['一级行业'] = df['industry'].apply(translate).str.replace('电讯','科技').str.replace('资讯科技','科技')
    df['二级行业'] = df['business'].apply(translate)
    df['三级行业'] = df['child-business'].apply(translate)
    df['证券名称'] = df['name'].apply(translate)
    df['证券代码'] = df['stock-id']
    df['总市值'] = df['market-cap']
    return df

def get_hkex_df():
    token = get_token()
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'}
    url = 'https://www1.hkex.com.hk/hkexwidget/data/getequityfilter?lang=chi&token='+token+'&sort=5&order=0&all=1&qid=NULL&callback=NULL'

    res = requests.get(url, headers=headers)
    hkex_df = pd.DataFrame(json.loads(res.text.replace('NULL(','').replace(')',''))['data']['stocklist'])
    hkex_df['证券代码'] = hkex_df['ric'].apply(format_code)
    hkex_df['涨跌幅'] = pd.to_numeric(hkex_df['pc'], errors="coerce")
    return hkex_df

hk_industry_df = get_industry_df()
hkex_df = get_hkex_df()

df = hk_industry_df.merge(hkex_df, on = '证券代码', how = 'inner')
df = df[~df['一级行业'].isnull()]
df['总市值'] = (df['总市值']/100000000).round(1).fillna(0) 
df = df[~df['涨跌幅'].isnull()]
df = df[df['总市值'] > 0]

fig = px.treemap(df, 
                 path=['一级行业','二级行业','三级行业','证券名称'],  # 指定层次结构，每一个层次都应该是category型的变量
                 values='总市值', # 需要聚合的列名
                 color='涨跌幅', 
                 custom_data=['涨跌幅','证券代码','总市值','ls'],
                 range_color = [-10, 10], # 色彩范围最大最小值
                 hover_data= {'涨跌幅':':.2',
                             '总市值':':.1f'}, # 鼠标悬浮显示数据的格式
                 color_continuous_scale=["seagreen",'lightgrey', "indianred"],
                 height = 900,
                #  width = 1600,
                 color_continuous_midpoint=0 , # 颜色变化中间值设置为增长率=0
                )
fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.update_coloraxes(showscale=False)
fig.update_traces(textposition='middle center', 
                  textinfo='label', 
                  textfont = dict(color='white'),
                  texttemplate= "%{label}<br>%{customdata[0]:.2f}%<br>",
                  hovertemplate= "%{customdata[1]}<br>%{label}<br>%{customdata[3]:.2f}  (%{customdata[0]:.2f}%)<br>总市值=%{customdata[2]:d}亿") # 显示企业名称和市值，字体24

fig.show()